# Finetune LLaMA-3 on AbstRCT dataset for ATC

## Libraries

In [1]:
# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd ../LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

/bin/bash: line 1: git: command not found
/nfs/scratch/umushtaq/am_work/LLaMA-Factory
CITATION.cff  Makefile      assets/  evaluation/     requirements.txt  src/
LICENSE       README.md     data/    examples/       scripts/          tests/
MANIFEST.in   README_zh.md  docker/  pyproject.toml  setup.py


/home/umushtaq/.conda/envs/notebook/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///nfs/scratch/umushtaq/am_work/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached gradio-4.37.2-py3-none-any.whl.metadata (15 kB)
  Using cached gradio_client-1.0.2-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.0 MB/s eta 0:00:00a 0:00:01
Using cached gradio-4.37.2-py3-none-any.whl (12.3 MB)
Using cached gradio_client-1.0.2-py3-none-any.whl (318 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 8.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.2 MB/s eta 0:00:0000:0100:01
Checking if build backend supports build_editable ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.8.3.dev0-0.editable-py3-none-any.whl si

In [3]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [2]:
# !nvidia-smi

Fri Jul  5 16:16:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 PCIe               Off |   00000000:82:00.0 Off |                    0 |
| N/A   60C    P0             94W /  350W |       1MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
import os
import ast
import json
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report

In [6]:
os.getcwd()

'/nfs/scratch/umushtaq/am_work/notebooks'

In [7]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Load Dataset

In [8]:
os.getcwd()

'/nfs/scratch/umushtaq/am_work/notebooks'

In [9]:
# *** MODEL NAME ***

base_model = "unsloth/llama-3-70b-bnb-4bit"

# with open("tmp.pkl", "rb") as fh:
        
#         l = pickle.load(fh)
#         base_model = l[0]
#         train_dataset_name = l[1]
#         test_dataset_name = l[2]

In [10]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dataset_dir = os.path.join(parent_dir, "datasets")

# *** TRAIN DATASET NAME *** #

train_dataset_name = "PE_ATC_essay_train.json"
train_dataset_file = os.path.join(dataset_dir, train_dataset_name)

# *** TEST DATASET NAME *** #

test_dataset_name = "PE_ATC_essay_test.json"
test_dataset_file = os.path.join(dataset_dir, test_dataset_name)

In [11]:
print(base_model, train_dataset_name, test_dataset_name, train_dataset_file, test_dataset_file)

unsloth/llama-3-8b-Instruct-bnb-4bit PE_ATC_essay_train.json PE_ATC_essay_test.json /nfs/scratch/umushtaq/am_work/datasets/PE_ATC_essay_train.json /nfs/scratch/umushtaq/am_work/datasets/PE_ATC_essay_test.json


## Fine-tune Model

In [12]:
# *** MODEL DIR ***
model_name = f"""{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}"""

train_file = os.path.join(parent_dir, f"""train_file_args/{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}.json""")
output_dir = os.path.join(parent_dir, "models", model_name)

nb_epochs = 50 # 3

In [13]:
# create new dirs for output and train file

In [14]:
model_name

'PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit'

In [15]:
train_file

'/nfs/scratch/umushtaq/am_work/train_file_args/PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit.json'

In [16]:
output_dir

'/nfs/scratch/umushtaq/am_work/models/PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit'

In [17]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [18]:
dataset_info_line

{'file_name': '/nfs/scratch/umushtaq/am_work/datasets/PE_ATC_essay_train.json',
 'columns': {'prompt': 'instruction', 'query': 'input', 'response': 'output'}}

In [19]:
os.getcwd()

'/nfs/scratch/umushtaq/am_work/notebooks'

In [20]:
with open(os.path.join(parent_dir, "LLaMA-Factory/data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["persuasive_essays"] = dataset_info_line

with open(os.path.join(parent_dir, "LLaMA-Factory/data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

In [23]:
args = dict(
  stage="sft",                           # do supervised fine-tuning
  do_train=True,
  model_name_or_path=base_model, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  cache_dir="/blabla",
  # dataset="persuasive_essays",           # use alpaca and identity datasets
  dataset="persuasive_essays",           # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir=output_dir,                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=3000,                       # save checkpoint every 1000 steps
  learning_rate=5e-5,                    # the learning rate
  num_train_epochs=nb_epochs,            # the epochs of training
  max_samples=2000,                       # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                    # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  fp16=True,                             # use float16 mixed precision training
  report_to="none"                       # discards wandb
)

In [24]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [25]:
os.getcwd()

'/nfs/scratch/umushtaq/am_work/notebooks'

In [26]:
%cd ../LLaMA-Factory

/nfs/scratch/umushtaq/am_work/LLaMA-Factory


/home/umushtaq/.conda/envs/notebook/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [27]:
!set train_file = train_file

In [28]:
train_file

'/nfs/scratch/umushtaq/am_work/train_file_args/PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit.json'

In [29]:
!llamafactory-cli train $train_file

07/02/2024 18:35:40 - INFO - llamafactory.cli - Initializing distributed tasks at: 127.0.0.1:24436
W0702 18:35:41.324000 140183078822400 torch/distributed/run.py:757] 
W0702 18:35:41.324000 140183078822400 torch/distributed/run.py:757] *****************************************
W0702 18:35:41.324000 140183078822400 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0702 18:35:41.324000 140183078822400 torch/distributed/run.py:757] *****************************************
07/02/2024 18:35:48 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/02/2024 18:35:48 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
07/02/2024 18:35:48 - INFO - llamafactory.hparams.parser - Process r

In [ ]:
os.getcwd()

## Inference on the fine-tuned model

In [ ]:
output_dir

In [ ]:
os.listdir(output_dir)

In [ ]:
# %cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path=base_model, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=output_dir,            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)


In [ ]:
model = ChatModel(args)

In [ ]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

In [ ]:
test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append("\nUser:" + sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [ ]:
test_predictions = []

for prompt in tqdm(test_prompts):

    messages = []
    messages.append({"role": "user", "content": prompt})

    response = ""
    
    for new_text in model.stream_chat(messages):
        #print(new_text, end="", flush=True)
        response += new_text
        #print()
    test_predictions.append({"role": "assistant", "content": response})

    torch_gc()

In [ ]:
# next(model.engine.model.parameters()).is_cuda

In [ ]:
# len(test_predictions)

In [ ]:
# test_predictions

In [ ]:
with open(os.path.join(output_dir, f"""PE_ATC_results_{nb_epochs}.pickle"""), 'wb') as fh:
    results_d = {"ground_truths": test_grounds,
                 "predictions": test_predictions    
        
    }
    pickle.dump(results_d, fh)

## Post-processing

In [ ]:
# import pickle
# import os
# import pandas as pd
# import json
# import ast
# from sklearn.metrics import classification_report

In [ ]:
#'/Utilisateurs/umushtaq/models/PE_ATC_llama-3-8b-Instruct-bnb-4bit'
# output_dir = 'models/PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit'
# nb_epochs = 10

In [ ]:
with open(os.path.join(output_dir, f"""PE_ATC_results_{nb_epochs}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [ ]:
grounds = results["ground_truths"]

In [ ]:
preds = results["predictions"]

In [ ]:
grounds = [json.loads(x)["component_types"] for x in grounds]

In [ ]:
preds = [x["content"] for x in preds]

In [ ]:
preds = [json.loads(x)["component_types"] for x in preds]

In [ ]:
def opposite(component_type):

    if component_type == "Premise":
        return "Claim"
    elif component_type == "Claim":
        return "Premise"
    elif component_type == "MajorClaim":
        return "Claim"

In [ ]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [ ]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [ ]:
ATC_preds = [item for row in preds for item in row]
ATC_grounds = [item for row in grounds for item in row]

In [ ]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

## Results

In [ ]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

In [ ]:
with open(f"""{output_dir}/classification_report.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(ATC_grounds, ATC_preds, output_dict=True), fh)

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()